In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
import random
p = 0.001  # 1% of the lines
# keep the header, then take only 10% of lines
# if random from [0,1] interval is greater than 0.1 the row will be skipped
df = pd.read_csv(
         'learning_traces.13m.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df.drop_duplicates(inplace=True)
df.info()

#### feature engineering

In [ ]:
# before we try to learn good values for theta 
# we need to construct x

# x = information a students history learning a certain word

df.iloc[0,:]

In [ ]:
# (df['p_recall'] == (df['session_correct'])/(df['session_seen'])).sum() == df.shape[0]

# p_recall is the ratio of session_correct/session_seen

# p_recall is "y" "ground truth"

# predicted_p_recall is "y_hat" "prediction"

# error(p_recall,predicted_p_recall) <- we want this to be as small as possible

# if we can very reliably predict p_recall, what is the value of this in real-life terms?


In [ ]:
# get parts of speech

def lexeme_df(filename):

    import re
    df_single_col = pd.read_csv(filename, delimiter='\t', header=None, names=['line'])

    def split_line(line):
        parts = re.split(r'\s+', line, maxsplit=2)
        if len(parts) == 3:
            return parts
        return [None, None, None]

    df_split = df_single_col['line'].apply(split_line)
    df = pd.DataFrame(df_split.tolist(), columns=['lexeme', 'category', 'meaning'])

    return df

In [ ]:
lexemes = lexeme_df('lexeme_reference.txt')
lexemes.head()

In [ ]:
lexemes['lexeme'].nunique()

In [ ]:
# one-hot encoding
# dummy variables / indicator variables

# df.loc[0,"lexeme_string"]

"<det><def><nt><sg><nom>"

# det, df, nt, sg, nom + 87 more 

# word | det | def | nt | sg | nom | ...
# das  |  1  | 1   | 1  | 1  |  1  | 0 ...



# look for the first <, remove everything to the left
# then remove <, >

# 

def extract_right_of_lt(text):
    import re
    match = re.search(r'<(.*)', text)
    return match.group(1) if match else ''




In [ ]:
df['lexeme_string'] = df['lexeme_string'].apply(extract_right_of_lt)
df['lexeme_string'] = df['lexeme_string'].str.replace("<"," ")
df['lexeme_string'] = df['lexeme_string'].str.replace(">","")
df['lexeme_string'] = df['lexeme_string'].str.replace("*","")
df['lexeme_string'] = df['lexeme_string'].str.replace("/","")
df['lexeme_string']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), binary=True)
vectorized_words = vectorizer.fit_transform(df['lexeme_string'])
vectorized_df = pd.DataFrame(vectorized_words.toarray(), columns=vectorizer.get_feature_names_out(), index=df.index)

In [ ]:
# no longer optimizing this for now,
# i want to talk about specifying the model
# we'll return to this

vectorized_df.head()

In [ ]:
vectorized_df.sum().sort_values()

In [ ]:
# curious about most common parts of speech.
# how different parts of speech correspond to rate of correct responses

#
vectorized_df.sum().sort_values(ascending=False)[0:20].plot(kind = "bar",title="most common parts of speech")
plt.show()


In [ ]:
# p_recall 

df = pd.concat([df,vectorized_df],axis=1)
df.shape

In [ ]:
df.drop(columns = ['lexeme_string'],inplace=True)

In [ ]:
grp_by_columns = list(df.columns[~df.columns.isin(['p_recall','delta','user_id','timestamp',"history_seen", "history_correct", "session_seen",  "session_correct"])])

"""
select avg(p_recall)
from df
group by var1

select avg(p_recall)
from df
group by var2

...

select avg(p_recall)
from df
group by varn

"""

"""

select avg(p_recall)
from df
group by var1, ..., varn

"""

# 1515 binary vectors (one for each part of speech)

# 0 avg(p_recall)
# 1 avg(p_recall)

# df.drop(columns = grp_by_columns).mean(axis=1)



# df[grp_by_columns].mean(axis = 1)

list_of_recall_variation_by_column = []

for col in grp_by_columns:

    variation_of_means = df.groupby(col)['p_recall'].mean().std()

    variation_dict = {"column_name":col,
                      "variation_of_means":variation_of_means}

    list_of_recall_variation_by_column.append(variation_dict)

recall_variation_by_column = pd.DataFrame(list_of_recall_variation_by_column)
    
# col_name, std of group means

#### fitting

In [ ]:
# should start with a really simple feature vector
# (history_seen, history_correct)

# decision: recode delta to days (that's what duolingo did)
# round to one decimal point

simple_df = df[['p_recall','delta','history_seen','history_correct']]
simple_df["delta_days"] = np.round(simple_df.loc[:,"delta"].copy()/(60*60*24),1)
simple_df.drop(columns = ["delta"],inplace=True)
simple_df.loc[simple_df["p_recall"] == 0,"p_recall"] = simple_df.loc[simple_df["p_recall"] == 0,"p_recall"] + 1e-3

# h
# -delta/log2(p)
simple_df["h"] = -1*simple_df["delta_days"]/(np.log2(simple_df["p_recall"]) + 1e-3)

simple_df.sample(5)
# input_dim = 2

In [ ]:
# h_hat = 2**(theta*x)

# theta = [1,3]
# x = [10,5]

# 10 + 15 = 25

# h_hat = 2**25
# predicted_p_recall = 2**(-1*(2/(2**25))) = 1

# we need an error function to quantify how wrong we are

# (p_recall - predicted_p_recall)**2 <- error
# predicted_p_recall = 2**(-1*(delta/predicted_half_life))
# predicted_half_life = 2**(theta*x)


# given
# p_recall is given
# delta is given
# x is given

# we don't have theta

# we are going initialize theta with some random numbers
# so then we have theta

# once we have theta we can calculate the error
# and start learning

# to-do

# write formulas for predicted half_life and predicted p_recall
# write formula for loss function
# and import an optimizer 
# run the optimizer on the loss function + our data

In [ ]:
# going to use pytorch
# we are using custom loss function and our model is not one of the standard ML models, like linear regression 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# create a fresh instance of SpacedRepetition model
# what happens 
# mymodel = SpacedRepetition(2)
# under the hood
# theta = [theta1,theta2]

In [ ]:
class SpacedRepetition(nn.Module):
    
    def __init__(self, input_dim, alpha, lambda_reg):

        super(SpacedRepetition, self).__init__()
        self.theta = nn.Linear(input_dim, 1, bias=False)
        self.alpha = alpha
        self.lambda_reg = lambda_reg

    def forward(self, x):
        # estimating h_hat = 2^(theta . x)
        theta_x = self.theta(x)  # dot product of theta and x
        h_hat = torch.pow(2, theta_x)
        return h_hat

    def prediction(self, h_hat, delta):
        p_hat = torch.pow(2, -1 * (delta / h_hat))
        return p_hat

    def loss(self,p, p_hat, h, h_hat):
        loss_p = torch.sum((p_hat - p) ** 2)
        loss_h = torch.sum((h_hat - h) ** 2)
        reg_term = self.lambda_reg * torch.sum(self.theta.weight ** 2)
        total_loss = loss_p + self.alpha * loss_h + reg_term
        return total_loss


In [ ]:
# object oriented programming

# python is a very flexible programming language

# one thing it lets you do is define things called Classes

# suppose you are a game developer
# designing a world for your game
# and your world has trees
# you write a tree class
# tree class defines what attributes trees can have
# tree: height, color, bears_fruit, number of leaves, ...

# tree_A = tree(height = 100, color = green, bears_fruit = false, number of leaves = 1800)

# tree class exists
# new class called MagicTrees

# class MagicTree(Tree):
# super(MagicTree, self).__init__()

# MagicTree is called a sub-class of Tree
# Tree is a superclass of MagicTree

# linear regression is implemented as a class
# result = smf.ols("y ~ x",data=df)
# result.summary
# result.params
# ....

# class SpacedRepetition(nn.Module)
# super(SpacedRepetition, self).__init__()

# inheritance


# __init__(self, input_dim) "dunder method" "double underscore method"

# mymodel = SpacedRepetition(2)
# np.dot(mymodel.theta.weight[0].detach().numpy(),simple_df[['history_seen','history_correct']].iloc[0,:])

In [ ]:
# .317

# 2**(.317)
# predicted_half_life = 1.24
# predicted_p_recall = 2**(-1*(83/1.24)) = 0
# real p_recall = 1

# (real p_recall - predicted p recall)**2 = 1

# after optimization
# hopefully we have thetas that produce a predicted p recall that is closer to the truth

# forward pass

# 128,000 records
# each has history_seen, history_correct
# x_i = [history_seen, history_correct] for i in range(128000)
# y_i = p recall for ith person
# using theta and x_i we take a dot product theta*x_i = z_i
# predicted_hat_i = 2**(z_i)
# delta is the last time (in days) that they saw the word
# predicted_p_recall_i = 2**(-1*(delta/predicted_hat_i))
# predicted_p_recall_i is our "final output"
# now we do it for i = 1, ..., i = 128,000
# measure how wrong we are
# (p_recall_i - predicted_p_recall_i)**2 <- there's more to the error function but this what makes it go
# add it up for i = 1 , ..., i = 128,000
# ???????? optimization
# we have a new theta - new theta will hopefully yield smaller error
# rinse and repeat until error stops going down.
# then you have your model.

In [ ]:
# half_life = 2**(theta*x)
#
# simple_df[['history_seen','history_correct']].iloc[0,:]
# df.loc[0,"delta"]/60/60/24
# df.loc[0,'p_recall']

In [ ]:
# training the model

# reminder of the training process

# step 1: we pass all of our data through the model to get predicted h_hat
# we will have 129,228 predictions for h_hat
# will use these h_hats to get 129,228 predictions for p_recall
# we will check these predicted p_recalls against observed p_recall (predicted_p_recall - p_recall)**2
# we add up these squared errors across all 129,228 rows
# then we update model parameters (we will use a in-built or provided optimizer) (we will not get into the details)
# then we start again at the top, at step 1:
# we do this many times, until our error no longer improves.

In [ ]:
def standardize_torch_vector(vec):
    mean = vec.mean(dim=0, keepdim=True)
    std = vec.std(dim=0, keepdim=True)
    vec_standardized = (vec - mean) / std
    return vec_standardized

def minmax_torch_vector(vec):
    min_ = vec.min(dim=0)
    max_ = vec.max(dim=0)
    vec_minmax = (vec - min_.values)/(max_.values - min_.values)
    return vec_minmax

In [ ]:
input_dim = 2
learning_rate = 0.001
num_epochs = 100 
alpha = 0.01
lambda_reg = 0.01

model = SpacedRepetition(input_dim, alpha, lambda_reg)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

x = minmax_torch_vector(torch.tensor(simple_df[['history_seen', 'history_correct']].values, dtype=torch.float32))
p = minmax_torch_vector(torch.tensor(simple_df['p_recall'].values, dtype=torch.float32))
h = minmax_torch_vector(torch.tensor(simple_df['h'].values, dtype=torch.float32))
delta = minmax_torch_vector(torch.tensor(simple_df['delta_days'].values, dtype=torch.float32))


In [ ]:
losses = []
for epoch in range(num_epochs):
    model.train()
    
    h_hat = model.forward(x)
    p_hat = model.prediction(h_hat, delta)

    loss = model.loss(p, p_hat, h, h_hat)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
plt.plot(range(num_epochs), losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs. Epochs')
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    h_hat = model(x)